In [6]:
import numpy as np
from numba import njit

In [49]:
def get_N_primes(N):
    '''
    N (int > 1): how many of the first prime numbers to return.
    '''
    
    sieve = np.full(N+1, True)
    
    for i in range(2, int(np.sqrt(N+1))):
        if sieve[i]:
            
            j = i**2
            n = 0
            while j < N:
                print(j)
                sieve[j-2] = False
                n += 1
                j = i**2 + n*i
                
    return sieve    

In [50]:
get_N_primes(10)

4
6
8


array([ True,  True, False,  True, False,  True, False,  True,  True,
        True,  True])

In [42]:
get_N_primes(5)

array([ True,  True,  True,  True,  True])

In [26]:
np.arange(2, 12)

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [35]:
get_N_primes(10)

array([ True,  True,  True,  True, False,  True, False,  True, False,
        True,  True])

In [38]:
np.arange(2, 13)[get_N_primes(10)]

array([ 2,  3,  4,  5,  7,  9, 11, 12])

In [14]:
get_N_primes(10)

array([ True,  True,  True,  True, False,  True, False,  True, False,
        True])

In [8]:
np.full(10, True)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])